In [ ]:
# Copyright 2022 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# GCP上的端到端机器学习：MLOps阶段1：形式化：使用Vertex AI数据标注开始

<table align="left">
  <td>
    <a href="https://colab.research.google.com/github/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/community/ml_ops/stage1/get_started_with_data_labeling.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/colab-logo-32px.png" alt="Colab logo"> 在Colab中运行
    </a>
  </td>
  <td>
    <a href="https://github.com/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/community/ml_ops/stage1/get_started_with_data_labeling.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/github-logo-32px.png" alt="GitHub logo">
      在GitHub上查看
    </a>
  </td>
  <td>
    <a href="https://console.cloud.google.com/vertex-ai/workbench/deploy-notebook?download_url=https://raw.githubusercontent.com/GoogleCloudPlatform/vertex-ai-samples/main/notebooks/community/ml_ops/stage1/get_started_with_data_labeling.ipynb">
      <img src="https://lh3.googleusercontent.com/UiNooY4LUgW_oTvpsNhPpQzsstV5W8F7rYgxgGBD85cWJoLmrOzhVs_ksK_vgx40SHs7jCqkTkCk=e14-rj-sc0xffffff-h130-w32" alt="Vertex AI logo">
      在Vertex AI Workbench中打开
    </a>
  </td>
</table>
<br/><br/><br/>

## 概述

本教程演示了如何在Google Cloud上使用Vertex AI进行端到端MLOps生产。本教程涵盖了第1阶段：数据管理：开始使用Vertex AI数据标记服务。

### 目标

在本教程中，您将学习如何使用`Vertex AI数据标记`服务。

本教程使用以下Google Cloud ML服务：

- `Vertex AI数据标记`
- `Vertex AI数据集`

执行的步骤包括：

- 为数据标记者创建专家池。
- 创建数据标记工作。
- 提交数据标记工作。
- 列出数据标记工作。
- 取消数据标记工作。

了解更多关于[请求Vertex AI数据标记工作](https://cloud.google.com/vertex-ai/docs/datasets/data-labeling-job)。

数据集

本教程使用的数据集是来自TensorFlow数据集的[花卉数据集](https://www.tensorflow.org/datasets/catalog/tf_flowers)。在本教程中使用的数据集版本存储在公共云存储桶中。训练模型可以预测一幅图像属于五种花朵中的哪一种：雏菊、蒲公英、玫瑰、向日葵或郁金香。

费用

本教程使用Google Cloud的计费组件：

* Vertex AI
* Cloud Storage

了解[Vertex AI 价格](https://cloud.google.com/vertex-ai/pricing)和[Cloud Storage 价格](https://cloud.google.com/storage/pricing)，并使用[Pricing 计算器](https://cloud.google.com/products/calculator/)根据您预计的使用情况生成成本估算。

## 安装

安装以下包以执行这个笔记本。

In [ ]:
import os

# The Vertex AI Workbench Notebook product has specific requirements
IS_WORKBENCH_NOTEBOOK = os.getenv("DL_ANACONDA_HOME") and not os.getenv("VIRTUAL_ENV")
IS_USER_MANAGED_WORKBENCH_NOTEBOOK = os.path.exists(
    "/opt/deeplearning/metadata/env_version"
)

# Vertex AI Notebook requires dependencies to be installed with '--user'
USER_FLAG = ""
if IS_WORKBENCH_NOTEBOOK:
    USER_FLAG = "--user"

# Install the packages
! pip3 install --upgrade google-cloud-aiplatform $USER_FLAG -q
! pip3 install --upgrade google-cloud-storage $USER_FLAG -q

### 重新启动内核

一旦您安装了Vertex AI SDK和Google *cloud-storage*，您需要重新启动笔记本内核，以便它可以找到这些软件包。

In [ ]:
import os

if not os.getenv("IS_TESTING"):
    # Automatically restart kernel after installs
    import IPython

    app = IPython.Application.instance()
    app.kernel.do_shutdown(True)

## 开始之前

### GPU 运行时

*如果您有此选项，请确保在 GPU 运行时中运行此笔记本。在 Colab 中，选择* **运行时 > 更改运行时类型 > GPU**

### 设置您的 GCP 项目

**以下步骤是必需的，无论您的笔记本环境如何。**

1. [选择或创建一个 GCP 项目](https://console.cloud.google.com/cloud-resource-manager)。当您首次创建账户时，您将获得 $300 的免费信用以支付计算/存储成本。

2. [确保为您的项目启用了计费。](https://cloud.google.com/billing/docs/how-to/modify-project)

3. [启用 Vertex AI APIs 和 Compute Engine APIs。](https://console.cloud.google.com/flows/enableapi?apiid=aiplatform.googleapis.com,compute_component)

4. [Google Cloud SDK](https://cloud.google.com/sdk) 已经安装在 Vertex AI Workbench Notebooks 中。

5. 在下面的单元格中输入您的项目 ID。然后运行该单元格，以确保 Cloud SDK 在本笔记本中的所有命令中使用正确的项目。

**注意**: Jupyter 会将以 `!` 为前缀的行视为 shell 命令，并将以 `$` 为前缀的 Python 变量插入这些命令中。

### 设置您的项目ID

**如果您不知道您的项目ID**，您可以使用 `gcloud` 命令获取您的项目ID。

In [ ]:
PROJECT_ID = "[your-project-id]"  # @param {type:"string"}

In [ ]:
if PROJECT_ID == "" or PROJECT_ID is None or PROJECT_ID == "[your-project-id]":
    # Get your GCP project id from gcloud
    shell_output = !gcloud config list --format 'value(core.project)' 2>/dev/null
    PROJECT_ID = shell_output[0]
    print("Project ID:", PROJECT_ID)

In [ ]:
! gcloud config set project $PROJECT_ID

区域

您还可以更改`REGION`变量，该变量用于整个笔记本的操作。以下是Vertex AI支持的区域。我们建议您选择离您最近的区域。

- 美洲：`us-central1`
- 欧洲：`europe-west4`
- 亚太：`asia-east1`

您可能无法使用多区域存储桶进行Vertex AI的训练。并非所有区域都支持所有Vertex AI服务。 

了解更多关于[Vertex AI区域](https://cloud.google.com/vertex-ai/docs/general/locations)。

In [ ]:
REGION = "[your-region]"  # @param {type: "string"}

if REGION == "[your-region]":
    REGION = "us-central1"

时间戳

如果您正在参加直播教程会议，您可能正在使用共享测试帐户或项目。为了避免用户在创建的资源上发生名称冲突，您可以为每个实例会话创建一个时间戳，并附加到在本教程中将要创建的资源的名称上。

In [ ]:
from datetime import datetime

TIMESTAMP = datetime.now().strftime("%Y%m%d%H%M%S")

邮箱

您需要一个邮箱地址来发送标注任务请求。这个邮箱地址将是数据标注专家池的管理员。

在本教程中，如果您没有指定一个邮箱地址，将使用与您的项目ID关联的邮箱地址。

In [ ]:
EMAIL = "[your-email-address]"  # @param {type: "string"}

In [ ]:
if EMAIL == "[your-email-address]":
    shell_output = ! gcloud auth list 2>/dev/null
    EMAIL = shell_output[2].replace("*", "").strip()

print(EMAIL)

### 认证您的 Google Cloud 账户

**如果您正在使用 Vertex AI 工作台笔记本**，您的环境已经得到了认证。

**如果您正在使用 Colab**，运行下面的单元格，并按照提示进行身份验证以通过 oAuth 认证您的账户。

**否则**，请按照以下步骤操作：

1. 在 Cloud 控制台中，转到 [**创建服务帐户密钥** 页面](https://console.cloud.google.com/apis/credentials/serviceaccountkey)。

2. 点击 **创建服务帐户**。

3. 在 **服务帐户名称** 字段中输入一个名称，并
   点击 **创建**。

4. 在 **授予此服务帐户对项目的访问权限** 部分，点击 **角色** 下拉列表。在筛选框中输入 "Vertex AI"，
   选择 **Vertex AI 管理员**。在筛选框中输入 "Storage Object Admin"，选择 **Storage Object Admin**。

5. 点击 *创建*。一个包含您密钥的 JSON 文件将下载到您的本地环境。

6. 在下面的单元格中将您的服务帐户密钥路径输入为
   `GOOGLE_APPLICATION_CREDENTIALS` 变量，并运行此单元格。

In [ ]:
# If you are running this notebook in Colab, run this cell and follow the
# instructions to authenticate your GCP account. This provides access to your
# Cloud Storage bucket and lets you submit training jobs and prediction
# requests.

import os
import sys

# If on Vertex AI Workbench, then don't execute this code
IS_COLAB = False
if not os.path.exists("/opt/deeplearning/metadata/env_version") and not os.getenv(
    "DL_ANACONDA_HOME"
):
    if "google.colab" in sys.modules:
        IS_COLAB = True
        from google.colab import auth as google_auth

        google_auth.authenticate_user()

    # If you are running this notebook locally, replace the string below with the
    # path to your service account key and run this cell to authenticate your GCP
    # account.
    elif not os.getenv("IS_TESTING"):
        %env GOOGLE_APPLICATION_CREDENTIALS ''

### 创建一个云存储桶

**无论您使用什么笔记本环境，都需要执行以下步骤。**

本教程旨在使用公共云存储桶中的训练数据，并为批量预测使用本地云存储桶。您也可以使用存储在本地云存储桶中的自己的训练数据。

在下面设置您的云存储桶的名称。它必须在所有云存储桶中是唯一的。

In [ ]:
BUCKET_NAME = "[your-bucket-name]"  # @param {type:"string"}
BUCKET_URI = f"gs://{BUCKET_NAME}"

In [ ]:
if BUCKET_URI == "" or BUCKET_URI is None or BUCKET_URI == "gs://[your-bucket-name]":
    BUCKET_NAME = PROJECT_ID + "aip-" + TIMESTAMP
    BUCKET_URI = "gs://" + BUCKET_NAME

只有当您的存储桶不存在时：运行以下单元格以创建您的Cloud Storage存储桶。

In [ ]:
! gsutil mb -l $REGION $BUCKET_URI

最后，通过检查云存储桶中的内容来确认对其的访问权限：

In [ ]:
! gsutil ls -al $BUCKET_URI

### 设置变量

接下来，设置一些在教程中使用的变量。
### 导入库并定义常量

#### 导入 Vertex AI SDK

将Vertex AI SDK导入到我们的Python环境中。

In [ ]:
import os
import sys
import time

import google.cloud.aiplatform as aip
from google.cloud import storage
from google.cloud.aiplatform import gapic
from google.protobuf.json_format import ParseDict
from google.protobuf.struct_pb2 import Value

### 初始化使用Python的Vertex AI SDK

为您的项目和对应的存储桶初始化使用Python的Vertex AI SDK。

In [ ]:
aip.init(project=PROJECT_ID, location=REGION)

#### Vertex AI常量

为Vertex AI设置以下常量：

- `API_ENDPOINT`：用于数据集、模型、作业、管道和端点服务的Vertex AI API服务端点。
- `PARENT`：用于数据集、模型和端点资源的Vertex AI位置根路径。

In [ ]:
# API Endpoint
API_ENDPOINT = "{}-aiplatform.googleapis.com".format(REGION)

# Vertex AI location root path for your dataset, model and endpoint resources
PARENT = "projects/" + PROJECT_ID + "/locations/" + REGION

常量架构

接下来，为与图像分类数据集相关的架构设置常量：

- 数据标签（注释）架构：告诉托管的数据集服务数据是如何标记（注释）的。

In [ ]:
# Image labeling task
LABELING_SCHEMA_IMAGE = "gs://google-cloud-aiplatform/schema/datalabelingjob/inputs/image_classification_1.0.0.yaml"

## 创建客户端

Vertex AI SDK采用客户端/服务器模型。在您的端上（Python脚本），您创建一个客户端，该客户端向服务器（Vertex AI）发送请求并接收响应。

在本教程中，您将使用多个客户端，请提前设置它们。

- 专业池服务用于专业池
- 作业服务用于数据标记

In [ ]:
# client options same for all services
client_options = {"api_endpoint": API_ENDPOINT}

clients = {}
clients["job"] = gapic.JobServiceClient(client_options=client_options)

# add client for specialist pool
clients["specialist_pool"] = gapic.SpecialistPoolServiceClient(
    client_options=client_options
)

for client in clients.items():
    print(client)

创建一个用于示例标记的CSV文件

接下来，您将为您要求标记的示例创建一个CSV文件。

在本示例中，要标记的示例是图片。在CSV文件的每一行中，您都要指定要标记的图片在云存储中的位置。

In [ ]:
test_filename = "labeling.csv"
LABELING_FILES = [
    "gs://cloud-samples-data/vision/automl_classification/flowers/daisy/100080576_f52e8ee070_n.jpg",
    "gs://cloud-samples-data/vision/automl_classification/flowers/daisy/102841525_bd6628ae3c.jpg",
]

IMPORT_FILE = BUCKET_URI + "/labeling.csv"

bucket = storage.Client(project=PROJECT_ID).bucket(BUCKET_URI.replace("gs://", ""))

# creating a blob
blob = bucket.blob(blob_name=test_filename)

# creating data variable
data = LABELING_FILES[0] + "\n" + LABELING_FILES[1] + "\n"

# uploading data variable content to bucket
blob.upload_from_string(data, content_type="text/csv")

# printing path of uploaded file
print(IMPORT_FILE)

# printing content of uploaded file
! gsutil cat $IMPORT_FILE

创建一个未标记的数据集

接下来，您需要创建一个数据集来标记数据。

In [ ]:
dataset = aip.ImageDataset.create("labeling_" + TIMESTAMP)
print(dataset)

## 导入未标记的数据

现在，将未标记的数据导入数据集，即要标记的示例。

In [ ]:
dataset.import_data(
    gcs_source=[IMPORT_FILE],
    import_schema_uri=aip.schema.dataset.ioformat.image.single_label_classification,
)

## 创建一个新的数据专家池

您的数据标注工作将会发送到一个数据专家池中。您可以拥有一个或多个专家池。

在下一步中，您可以通过方法 `create_specialist_pool()` 来创建一个新的专家池。请求应该包括以下参数：

- `name`：专家池的资源名称。
- `display_name`：专家池的可读名称。
- `specialist_manager_emails`：专家池经理（们）的电子邮件地址列表。

*注意：* 如果已经存在一个专家池，您可以使用现有的专家池。

In [ ]:
specialist_pool = {
    "name": "labeling_" + TIMESTAMP,
    "display_name": "labeling_" + TIMESTAMP,
    "specialist_manager_emails": [EMAIL],
}

request = clients["specialist_pool"].create_specialist_pool(
    parent=PARENT, specialist_pool=specialist_pool
)

result = request.result()
print(result)

specialist_name = result.name

specialist_id = specialist_name.split("/")[-1]

print(specialist_name)

## 创建数据标注任务

现在您已经有了专家池，您可以使用`create_data_labeling_job()`方法发送数据标注请求。

您的请求将包括以下内容：

- 包含未标记数据的Vertex AI数据集。
- 标注指令。

In [ ]:
# create placeholder file for instructions for data labeling
! echo "this is instruction" >> instruction.txt | gsutil cp instruction.txt $BUCKET_URI

In [ ]:
LABLEING_SCHEMA = LABELING_SCHEMA_IMAGE
INSTRUCTION_FILE = BUCKET_URI + "/instruction.txt"

inputs = ParseDict({"annotation_specs": ["rose"]}, Value())

data_labeling_job = {
    "display_name": "labeling_" + TIMESTAMP,
    "datasets": [dataset.resource_name],
    "labeler_count": 1,
    "instruction_uri": INSTRUCTION_FILE,
    "inputs_schema_uri": LABLEING_SCHEMA,
    "inputs": inputs,
    "annotation_labels": {
        "aiplatform.googleapis.com/annotation_set_name": "data_labeling_job_specialist_pool"
    },
    "specialist_pools": [specialist_name],
}

print(data_labeling_job)

request = clients["job"].create_data_labeling_job(
    parent=PARENT, data_labeling_job=data_labeling_job
)

print(request)

labeling_task_name = request.name

print(labeling_task_name)

### 获得数据标注工作

您可以使用`get_data_labeling_job()`方法获取有关您的数据标注工作的信息，以下是相应的参数：

- `name`： 标注任务的名称。

In [ ]:
request = clients["job"].get_data_labeling_job(name=labeling_task_name)
print(request)

取消数据标注任务

您可以使用`cancel_data_labeling_job()`方法取消数据标注请求，参数如下:

- `name`: 标注任务的名称。

In [ ]:
request = clients["job"].cancel_data_labeling_job(name=labeling_task_name)
print(request)

### 等待标记工作被取消

取消请求是异步的。下面的代码会循环检查标记工作的状态，直到状态变为已取消。

In [ ]:
while True:
    response = clients["job"].get_data_labeling_job(name=labeling_task_name)
    if response.state == gapic.JobState.JOB_STATE_CANCELLED:
        print("Labeling job CANCELED")
        break
    else:
        print("Canceling labeling job:", response.state)
        time.sleep(60)

清理工作

要清理此项目中使用的所有GCP资源，您可以删除用于本教程的[GCP项目](https://cloud.google.com/resource-manager/docs/creating-managing-projects#shutting_down_projects)。

否则，您可以删除本教程中创建的各个资源。

In [ ]:
# Set this to true only if you'd like to delete your bucket
delete_bucket = False

# Delete the dataset using the Vertex AI fully qualified identifier for the dataset
dataset.delete()

# Delete the labeling job using the Vertex AI fully qualified identifier for the dataset
request = clients["job"].delete_data_labeling_job(name=labeling_task_name)

# Delete the specialist pool using the Vertex AI fully qualified identifier for the dataset
clients["specialist_pool"].delete_specialist_pool(name=specialist_name)

# Delete the bucket created
if delete_bucket or os.getenv("IS_TESTING"):
    ! gsutil rm -r $BUCKET_URI